<a href="https://colab.research.google.com/github/bvogginger/NIR_Tutorial_at_NICE_2024/blob/main/notebooks/nir_to_brian2/NIR_to_brian2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# NIR to Brian2 demo: Synfire Chain

This notebook let's you look behind the scenes of NIR:

1. NIR Graph: its data structure and how to construct it
2. How to convert a NIR graph to a `brian2.Network` with brian2's neuron groups and synapses object

In [ ]:
! pip install nir
! pip install brian2

In [ ]:
import nir
import brian2
import matplotlib.pyplot as plt
import numpy as np

from numpy.random import default_rng

rng = default_rng()

## Synfire Chain
The [synfire chain](http://www.scholarpedia.org/article/Synfire_chains) is a feed-forward network of groups of many neurons. It is a standard benchmark and proof-of-concept for neuromorphic hardware.

In this demo we will use a simple synfire chain with the following architecture:
![Synfire Chain](images/schema_no_inh.png)

- An input population `stim` with 100 neuron creates a *spike volley* (each neuron spikes once with the spike time drawn from a Normal distribution).
- The `stim` population is connected to a group of neurons via excitatory connections. Synapses are created with a defined connection probability (e.g. 60 %)
- Each group of excitatory neurons `exc` with 100 neurons each is again connected to the following group of excitatory neurons with the same connections probability.
- Typically, the excitatory neurons only spike, if the neurons from the previous group spike synchronously (within a small time window, e.g. defined by the `sigma` of the Normal distribution
- For the first demo, we will use non-leaky Integrate & Fire neurons (IF), synaptic weights of `1.0` and spike threshold of `50.0` so that the neuron will fire once it has received 51 input spikes. *You may check the [NIR API documentation](https://nnir.readthedocs.io/en/latest/autoapi/nir/ir/neuron/index.html#nir.ir.neuron.IF) for the parameters and equation of the IF neuron.*

In [ ]:
n_groups = 5  # number of groups
n_nrns = 100  # neurons per group
p_conn = 0.6  # connection probability
weight = 1.0  # synaptic weight

# neuron params
R = 1.0
v_threshold = 50.0

def gen_weights_fixed_probability(n_pre: int, n_post:int, p:float, weight: float):
    """create a random weight matrix with fixed weight"""
    r = rng.random((n_post, n_pre))
    weights = np.zeros((n_post, n_pre), dtype=float)
    weights[r<p] = weight
    return weights

Create the NIR nodes (`Input`, `Linear`, `IF` and `Output`) for the synfire chain

In [ ]:
nodes = {} # dict of node names to NIR nodes
nodes["input"] = nir.Input(input_type=np.array([n_nrns]))
for i in range(n_groups):
    # Linear nodes
    nodes[f"linear_{i}"] = nir.Linear(weight=gen_weights_fixed_probability(n_nrns, n_nrns, p_conn, weight))
    
    # IF nodes
    nodes[f"if_{i}"] = nir.IF(r=np.full(n_nrns, R), v_threshold=np.full(n_nrns, v_threshold))

    # Output nodes for spike recording
    nodes[f"output_{i}"] = nir.Output(output_type=np.array([n_nrns]))

Create the directed edges

In [ ]:
edges = []  # directed edges defined as a list of tuples ("source_node", "target_node")

# Connection from input to first Linear node
edges.append(("input", "linear_0"))

# Edges between Linear and IF nodes
for i in range(n_groups):
    edges.append((f"linear_{i}", f"if_{i}"))

# Edges between IF nodes and Linear nodes of next group
for i in range(n_groups-1):
    edges.append((f"if_{i}", f"linear_{i+1}"))

# Edges from IF nodes to output nodes
for i in range(n_groups):
    edges.append((f"if_{i}", f"output_{i}"))

Create the NIR graph

In [ ]:
graph = nir.NIRGraph(nodes=nodes, edges=edges)

# Create brian2 network

Next we want to convert the NIR graph into a brian2 network

Create a `brian2.Network` and dictionairy to keep track of neuron groups, synapses and spike_monitors

In [ ]:
net = brian2.Network()
neuron_groups = {}
synapses = {}
spike_monitors = {}

## 1. Create NeuronGroups from NIR graph

Let's create NeuronGroups from `nir.IF` and `nir.Input` nodes.

- `IF` nodes are converted into `brian2.NeuronGroup` objects with matching neuron model equations.
- `Input` nodes are converted into `brian2.SpikeGeneratorGroup` objects

In [ ]:
# Loop over graph nodes and process IF and Input nodes
for name, node in graph.nodes.items():
    if isinstance(node, nir.IF):
        IF_eqn = """
        v : 1
        v_th : 1
        r : 1
        """

        shape = node.output_type["output"] # get output shape of NIR node
        # create a neuron group with IF model
        group = brian2.NeuronGroup(N=shape, model=IF_eqn, threshold="v>v_th", reset="v -= v_th")

        # set parameters
        group.r = node.r
        group.v_th = node.v_threshold

        neuron_groups[name] = group

    elif isinstance(node, nir.Input):
        shape = node.output_type["output"]  # get output shape of NIR node

        # create a spike generator group with empty spike times
        indices = np.array([], dtype=int)
        times = np.array([], dtype=float)*brian2.ms
        group = brian2.SpikeGeneratorGroup(shape, indices=indices, times=times)

        neuron_groups[name] = group

# add neuron groups to brian2 network
net.add(*neuron_groups.values())

## 2. Create Synapses from NIR graph

Let's create `brian2.Synapses` from `nir.Linear` nodes.

In [ ]:
# Loop over graph nodes and process IF and Input nodes
for name, node in graph.nodes.items():
    if isinstance(node, nir.Linear):
        # get weight matrix and reshape from (n_post, n_pre) to (n_pre, n_post)
        weight_matrix = node.weight.T

        # get source and target of Linear node by parsing the edges
        src_nodes = [src for (src, tgt) in graph.edges if tgt == name]
        tgt_nodes = [tgt for (src, tgt) in graph.edges if src == name]

        # make sure there is only 1 source and 1 target of the Linear node
        assert len(src_nodes) == 1
        assert len(tgt_nodes) == 1

        # get previously created source and target neuron groups
        src_group = neuron_groups[src_nodes[0]]
        tgt_group = neuron_groups[tgt_nodes[0]]

        # create synapses object
        syn = brian2.Synapses(src_group, tgt_group, model="w:1", on_pre="v += w")
        syn.connect() # connect full matrix
        syn.w = weight_matrix.flatten()

        synapses[name] = syn

# Add synapses to brian2 network
net.add(*synapses.values())

## 3. Create SpikeMonitors for Output notes

Let's use NIR's `Output` to set up the recording of the neuron groups with `brian2.SpikeMonitors`

In [ ]:
# create state monitors
for name, node in graph.nodes.items():
    if isinstance(node, nir.Output):
        # get source node by parsing the edges
        src_nodes = [src for (src, tgt) in graph.edges if tgt == name]
        assert len(src_nodes) == 1
        spike_mon = brian2.SpikeMonitor(neuron_groups[src_nodes[0]])
        spike_monitors[name] = spike_mon

# Add spike monitors to brian2 network
net.add(*spike_monitors.values())

## 5. Verify that no nodes were missed

Let's check if all nodes in the NIR graph have been processed

In [ ]:
# create a set with all the node names
node_names = set(graph.nodes.keys())

# remove nodes converted to neuron groups
for key in neuron_groups:
    node_names.remove(key)

# remove nodes converted to synapses objects
for key in synapses:
    node_names.remove(key)

# remove nodes converted to spike monitors
for key in spike_monitors:
    node_names.remove(key)

# Set of node names should be empty!
assert len(node_names) == 0

# Run simulation

Let's create input spikes

In [ ]:
input_group = neuron_groups["input"]

s_mean = 25.0
s_sigma = 2.0
input_spikes = rng.normal(s_mean, s_sigma, n_nrns)
indices = np.arange(n_nrns, dtype=int)
times = input_spikes*brian2.ms

input_group.set_spikes(indices, times)

# plot input spike volley
plt.plot(times/brian2.ms, indices, ".")
plt.xlim(0,50)
plt.xlabel("time [ms]")
plt.ylabel("input neuron")
plt.title("input spikes")

In [ ]:
# run brian2 simulation (this may take a while when you run brian for the 1st time)
net.run(50*brian2.ms, report="text")

In [ ]:
for i in range(n_groups):
    spike_mon = spike_monitors[f"output_{i}"]
    idx_offset = i*n_nrns
    plt.plot(spike_mon.t/brian2.ms, spike_mon.i + idx_offset, ".", label=f"group {i}")

plt.legend()
plt.xlim(0,50)
plt.xlabel("time [ms]")
plt.ylabel("neuron ID")